In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## 訓練データの読み込み


In [ ]:
train = pd.read_csv('../input/shelter-animal-outcomes/train.csv.gz')
train

## 統計情報
#### 仮説  
#### 選択する説明変数
立てた仮説として動物につけられた名前の有りなしで予測する結果「OutcomeType」に影響を与えると考えられる。  
そこで、新しい変数「has_name」を作成し、統計情報を確認したところ名前の有りなしでデータの偏りが大きく,予測する結果の分類に役立つと考えられる。    
また同様に、統計情報のからして、動物の年齢、動物の種類、動物の性別および去勢の有無は大きく結果に影響を与えると考えた。  
今回は「Name」「AnimalType」「SexuponOutcome」「AgeuponOutcome 」「Breed」「Color」を変換し、説明変数として選択した。  

In [ ]:
#「has_name」(名前が欠落している場合は0、存在する場合は1)
train_name = train
train_name['Name'] = train_name[['Name']].fillna(value=0)
train_name['has_name'] = (train_name['Name'] != 0).astype('int64')
train_name = train_name.drop('Name', axis=1)
plt.figure (figsize = (8,8))
sns.countplot(train_name['has_name'], hue = train_name['OutcomeType'])

In [ ]:
plt.figure (figsize = (8,8))
sns.countplot(train_name['AnimalType'], hue = train_name['OutcomeType'])

In [ ]:
plt.figure (figsize = (8,8))
sns.countplot(train_name['SexuponOutcome'], hue = train_name['OutcomeType'])

In [ ]:
#「AgeuponOutcome」数字＋英字から数値表記に直した「age_numeric」
train_name = train_name.apply(lambda x:x.fillna(x.value_counts().index[0]))
train_name.apply(lambda x: sum(x.isnull()/len(train_name)))
def age_converter(row):
    age_string = row['AgeuponOutcome']
    [age,unit] = age_string.split(" ")
    unit = unit.lower()
    if("day" in unit):
        if age=='0': return 1
        return int(age)
    if("week" in unit):
        if(age)=='0': return 7
        return int(age)*7
    elif("month" in unit):
        if(age)=='0': return 30
        return int(age) * 4*7
    elif("year" in unit):
        if(age)=='0': return 365
        return int(age) * 4*12*7
train_name['age_numeric'] = train_name.apply(age_converter, axis=1)
train_name = train_name.drop('AgeuponOutcome', axis=1)

plt.figure (figsize = (20,8))
sns.countplot(train_name['age_numeric'], hue = train_name['OutcomeType'])

In [ ]:
plt.figure(figsize = (8,8))
sns.boxplot(train_name['has_name'], train_name['age_numeric'], showfliers = False, hue = train_name['OutcomeType'])

In [ ]:
#「Color」を単色（０）と混色（１）に分類した変数「multi_colors」
train_name['multi_colors'] = train_name['Color'].apply(lambda x : 1 if '/' in x else 0)
plt.figure (figsize = (8,8))
sns.countplot(train_name['multi_colors'], hue = train_name['OutcomeType'])

In [ ]:
#「Breed」を'Mix'と'pure'に分類した変数「breed_type」
import re
train_name['breed_type'] = train_name.Breed.str.extract('({})'.format('|'.join(['Mix'])), 
                        flags=re.IGNORECASE, expand = False).str.lower().fillna('pure')
plt.figure (figsize = (8,8))
sns.countplot(train_name['breed_type'], hue = train_name['OutcomeType'])

## 欠損データの処理
「Name」と「OutcomeSubtype」の2つに欠損値が多数  
「OutcomeSubtype」は、予測しようとしている結果をさらに分類したものなので、予測には使わない  
「Name」は新しい変数「has_name」( 名前が欠落している場合は0、存在する場合は1)に変換する  
「SexuponOutcome」と「AgeuponOutcome」の欠落値はごくわずかなので最頻値で補完する

In [ ]:
#変数ごとに欠損しているデータ数を確認
train.apply(lambda x: sum(x.isnull()/len(train)))

In [ ]:
#訓練データから「OutcomeSubtype」を削除
train = train.drop('OutcomeSubtype', axis=1)

In [ ]:
#訓練データに「has_name」を追加し、「Name」を削除
train['Name'] = train[['Name']].fillna(value=0)
train['has_name'] = (train['Name'] != 0).astype('int64')
train = train.drop('Name', axis=1)

In [ ]:
#「SexuponOutcome」と「AgeuponOutcome」の欠損データを最頻値で補完
train = train.apply(lambda x:x.fillna(x.value_counts().index[0]))
train.apply(lambda x: sum(x.isnull()/len(train)))

欠損データの数が補完され、0になっているのがわかる

In [ ]:
#予測には役に立たないため「AnimalID」を削除
train = train.drop('AnimalID', axis=1)

## データの前処理
複雑なデータ内容を予測に使えるように訓練データを分類または数値化していく  
詳しくは参考サイト  
https://thefinance.jp/datascience/201109-2

In [ ]:
#それぞれの変数のカテゴリ数
columns = train.columns
for column in columns:
    print(column)
    print(train[column].nunique())

「Breed」はカテゴリ数が1380、「Color」はカテゴリ数が366であるので前処理が必要  
「Color」では混色と単色に分類する。  
「Breed」では'mix'という単語は雑種であると推測できるため、純血種と分類する  
「AgeuponOutcome」は数値表現にするため日数に変換する  
残りの変数はホットエンコーディングを使用して０，１の数値表記に変換する

In [ ]:
#「Color」を単色（０）と混色（１）に分類した変数「multi_colors」を訓練データに追加
train['multi_colors'] = train['Color'].apply(lambda x : 1 if '/' in x else 0)

In [ ]:
#「Breed」を'Mix'と'pure'に分類した変数「breed_type」を訓練データに追加
import re
train['breed_type'] = train.Breed.str.extract('({})'.format('|'.join(['Mix'])), 
                        flags=re.IGNORECASE, expand = False).str.lower().fillna('pure')

In [ ]:
#「AgeuponOutcome」数字＋英字から数値表記に直した「age_numeric」を追加し、「AgeuponOutcome」を削除
def age_converter(row):
    age_string = row['AgeuponOutcome']
    [age,unit] = age_string.split(" ")
    unit = unit.lower()
    if("day" in unit):
        if age=='0': return 1
        return int(age)
    if("week" in unit):
        if(age)=='0': return 7
        return int(age)*7
    elif("month" in unit):
        if(age)=='0': return 30
        return int(age) * 4*7
    elif("year" in unit):
        if(age)=='0': return 365
        return int(age) * 4*12*7
    
train['age_numeric'] = train.apply(age_converter, axis=1)
train = train.drop('AgeuponOutcome', axis=1)


In [ ]:
#上記の処理を行った訓練データ
train

In [ ]:
#データ処理前の変数「Breed」,「Color」, 「DateTime」を訓練データから削除
train = train.drop(['Breed','Color', 'DateTime'], axis=1)
#残った変数をホットエンコーディングを使用して０と１表記に変換
numeric_features = train.select_dtypes(include=['int64', 'float64']).columns
categorical_features = train.select_dtypes(include=['object']).drop(['OutcomeType'], axis=1).columns
dummy_columns = pd.get_dummies(train[categorical_features])
final_train = pd.concat([dummy_columns, train],axis=1)

#数値変換前の変数「AnimalType」,「breed_type」,「SexuponOutcome」,「top_colors」を訓練データから削除
final_train = final_train.drop(['AnimalType', 'breed_type', 'SexuponOutcome'], axis=1)

In [ ]:
#モデルの構築に使う訓練データ
final_train

## 予測モデルの構築
### 分析手法
ランダムフォレスト  
・予測精度が高い  
・特徴量の重要度が評価できる  
・過学習が起きにくい  
・複数のツリーの並列処理が可能  

詳しくは参考サイト  
https://www.stats-guild.com/analytics/12869  
https://qiita.com/Hawaii/items/5831e667723b66b46fba  
https://qiita.com/exp/items/1c6c9a3fae2d97bfa0c7  

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, log_loss
X = final_train.drop('OutcomeType', axis=1)
y = final_train['OutcomeType']

In [ ]:
#使用する機械学習のアルゴリズムはランダムフォレスト（Random forest）
clf = RandomForestClassifier(n_estimators=100, max_depth=9,
                            random_state=0)
rf_model = clf.fit(X, y)
train_predicted = clf.predict(X)

#訓練データを予測する

#Accuracyは予測した値と正解が一致していた数の単純なカウント(1に近づくほど優れたモデル)
print('Accuracy')
print(accuracy_score(train_predicted,y))

train_proba = rf_model.predict_proba(X)
#Log Lossは実際の結果からどのくらい違っていたのかより細かく分析できる(0に近づくほど優れたモデル)
#このコンペでのスコアの評価方法はLog Lossを用いている
print('Log Loss')
print(log_loss(y,train_proba))

In [ ]:
#モデルに適用した説明変数の影響度合いを可視化する
features=X.columns
importances = rf_model.feature_importances_
indices = np.argsort(importances)
plt.figure(figsize=(10,20))
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), features[indices])
plt.xlabel('Relative Importance')
plt.show

## テストデータの予測
訓練データと同じようにデータの前処理を行っているが一部異なる部分もある

In [ ]:
test = pd.read_csv('../input/shelter-animal-outcomes/test.csv.gz')
test

In [ ]:
#変数ごとに欠損しているデータ数を確認
test.apply(lambda x: sum(x.isnull()/len(test)))

In [ ]:
#「has_name」を追加し、「Name」を削除
test['Name'] = test[['Name']].fillna(value=0)
test['has_name'] = (test['Name'] != 0).astype('int64')
test = test.drop('Name', axis=1)

#「AgeuponOutcome」の欠損データを最頻値で補完
test = test.apply(lambda x:x.fillna(x.value_counts().index[0]))
test.apply(lambda x: sum(x.isnull()/len(test)))

#「ID」を削除
test = test.drop('ID', axis=1)

#「Color」を単色（０）と混色（１）に分類した変数「multi_colors」を訓練データに追加
test['multi_colors'] = test['Color'].apply(lambda x : 1 if '/' in x else 0)

#「Breed」を'Mix'と'pure'に分類した変数「breed_type」を訓練データに追加
test['breed_type'] = test.Breed.str.extract('({})'.format('|'.join(['Mix'])), 
                        flags=re.IGNORECASE, expand = False).str.lower().fillna('pure')

#「AgeuponOutcome」数字＋英字から数値表記に直した「age_numeric」を追加し、「AgeuponOutcome」を削除
test['age_numeric'] = test.apply(age_converter, axis=1)
test = test.drop('AgeuponOutcome', axis=1)

#データ処理前の変数「Breed」,「Color」,「DateTime」をを訓練データから削除
test = test.drop(['Breed','Color', 'DateTime'], axis=1)

#残った変数をホットエンコーディングを使用して０と１表記に変換
numeric_features_t = test.select_dtypes(include=['int64', 'float64']).columns
categorical_features_t = test.select_dtypes(include=['object']).columns
dummy_columns_t = pd.get_dummies(test[categorical_features_t])
final_test = pd.concat([dummy_columns_t, test],axis=1)

#数値変換前の変数「AnimalType」,「breed_type」,「SexuponOutcome」,を訓練データから削除
final_test = final_test.drop(['AnimalType', 'breed_type', 'SexuponOutcome'], axis=1)

In [ ]:
#予測に使うテストデータ
final_test

In [ ]:
#テストデータを予測モデルに適用する
X_test = final_test
test_proba = rf_model.predict_proba(X_test)

## 予測結果をコミット

In [ ]:
sub = pd.read_csv('../input/shelter-animal-outcomes/sample_submission.csv.gz')
sub

In [ ]:
sub.iloc[:,1:] = test_proba
sub

In [ ]:
sub.to_csv('submission.csv', index=False)

## スコア（Log Loss）
0.85211  

## 順位
815位/1599チーム  
## 反省点
今回予測に用いた説明変数では名前、年齢、性別、動物の種類の有無にが特に重要であることわかった。  
動物の色と種類に関してはデータ内容が複雑であったので「mix」、「pure」のような2つの単純な形に変換した。  
より精度を向上させるためにはこの色と種類を予測に使いやすく細かく分類したほうが良かったかもしれない。  
また、分析手法については今回はランダムフォレストを用いたが、  
勾配ブースティング決定木などのより高度なアルゴリズムとの比較を行えば、より精度を高めることが出来るかもしれない。  